# Python virtual environments

## What is a virtual environment?

A virtual environment is a Python environment such that the Python interpreter, libraries and scripts installed into it are isolated from those installed in other virtual environments, and (by default) any libraries installed in a “system” Python, i.e., one which is installed as part of your operating system.

## Why do we need virtual environments?

Imagine that you have an application that needs version 1 of LibFoo, but another application requires version 2. How can you use both these applications? If you install everything into /usr/lib/python2.7/site-packages (or whatever your platform’s standard location is), it’s easy to end up in a situation where you unintentionally upgrade an application that shouldn’t be upgraded.

In summary, virtual environments are a way of _creating isolated Python environments_ so that you can _work on a specific project_ without worrying about _affecting other projects_.

## How to create a virtual environment?

The module used to create and manage virtual environments is called venv. venv will usually install the most recent version of Python that you have available. If you have multiple versions of Python on your system, you can select a specific Python version by running python3 or whichever version you want.

To create a virtual environment, decide upon a directory where you want to place it, and run the venv module as a script with the directory path:

```bash
python3 -m venv tutorial-env
```

## How to activate a virtual environment?

Once you’ve created a virtual environment, you may activate it.

On Windows, run:
```bash
tutorial-env\Scripts\activate.bat
```

On Unix or MacOS, run:
```bash
source tutorial-env/bin/activate
```

This should change your prompt to the following:
```bash
(tutorial-env) $
```

The name of the current virtual environment will now appear on the left of the prompt to let you know that it’s active. From now on, any package that you install using pip will be placed in the tutorial-env folder, isolated from the global Python installation.

## What is this tutorial is about?

This tutorial is about how to create and manage virtual environments for Python projects. It will cover setting up a project environment and installing the required Python dependencies into an isolated environment. 

The goal is that at the end of this tutorial you will understand what is happening when you create and activate a virtual environment. Additionally, you will be able to make a script that creates a virtual environment and installs the required dependencies for your project, a skill that will be useful when you work on a cluster or a remote machine.

Note: What you will learn is actually what is happening under the hood of the [TAC](https://github.com/JeremieGince/TPAutoCorrect) package for autograding python homeworks.

In [ ]:
import os
import sys
import subprocess

 After importing the required packages, we will define a dictionary that will be used to get the path to the scripts folder of a virtual environment depending on the operating system.

In [ ]:
VENV_SCRIPTS_FOLDER_BY_OS = {
    "win32" : r"{}\Scripts",
    "linux" : "{}/bin",
    "darwin": "{}/bin",
}

We can now use the dictionary to get the path to the scripts folder of a virtual environment depending on the operating system.

In [ ]:
print(f"{sys.platform = }")
print(f"{VENV_SCRIPTS_FOLDER_BY_OS[sys.platform] = }")

We will now define a function that will run a command and print the stdout and stderr.

In [ ]:
def run_cmd(cmd):
    """
    Run a command and print the stdout and stderr.
    
    :Note: This function is equivalent to running the command in a terminal.
    
    :param cmd: The command to run.
    :type cmd: str
    :return: None
    """
    print(f"Command run: {cmd}")
    results = subprocess.run(cmd, stdout=subprocess.PIPE)
    stdout = "None" if results.stdout is None else results.stdout.decode('utf8')
    stderr = "None" if results.stderr is None else results.stderr.decode('utf8')
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")

Here, we will create a virtual environment named venv0 using the venv module and the run_cmd function.

In [ ]:
venv0_name = "venv0"
run_cmd(f"python -m venv {venv0_name}")

We will also create a virtual environment named venv1 to show that we can have multiple virtual environments.

In [ ]:
venv1_name = "venv1"
run_cmd(f"python -m venv {venv1_name}")

We can now see that the virtual environments have been created. After that we can get the path of the python executable of the virtual environments.

In [ ]:
venv0_scripts_path = VENV_SCRIPTS_FOLDER_BY_OS[sys.platform].format(venv0_name)
venv0_python_path = os.path.join(venv0_scripts_path, "python")
print(f"{venv0_python_path = }")

venv1_scripts_path = VENV_SCRIPTS_FOLDER_BY_OS[sys.platform].format(venv1_name)
venv1_python_path = os.path.join(venv1_scripts_path, "python")
print(f"{venv1_python_path = }")

We can now use the python executable of the virtual environments to install packages using pip. We will install pandas in venv0 and scikit-learn in venv1.

In [ ]:
run_cmd(f"{venv0_python_path} -m pip install pandas")

In [ ]:
run_cmd(f"{venv1_python_path} -m pip install scikit-learn")

We can now see that the packages have been installed in the virtual environments. In the next cell, we get the path of some python scripts that show if the packages have been installed correctly.

In [ ]:
file_using_pandas = "main_using_pandas.py"
file_using_sklearn = "main_using_sklearn.py"

We can now run the python files using the venv0 python executable. We can see that the file using pandas works but the file using scikit-learn does not work. It's expected because we only installed pandas in venv0.

In [ ]:
run_cmd(f"{venv0_python_path} {file_using_pandas}")
run_cmd(f"{venv0_python_path} {file_using_sklearn}")

This time, we will run the python files using the venv1 python executable. We can see that the file using scikit-learn works but the file using pandas does not work. It's expected because we only installed scikit-learn in venv1.

In [ ]:
run_cmd(f"{venv1_python_path} {file_using_pandas}")
run_cmd(f"{venv1_python_path} {file_using_sklearn}")

To be sure what packages are installed in a virtual environment, we can use the pip list command and see that the environments are indeed isolated and have only the packages that we installed.

In [ ]:
run_cmd(f"{venv0_python_path} -m pip list")

In [ ]:
run_cmd(f"{venv1_python_path} -m pip list")

## Using a class to manage virtual environments

We will now create a class that will help us manage virtual environments. The class will have the following methods:
- create: Create a virtual environment.
- run_py_cmd: Run a python command in the virtual environment.
- pip_install: Install a package in the virtual environment.
- get_pip_list: Get the list of packages installed in the virtual environment.
- \_\_repr\_\_: Return a string representation of the virtual environment.

The class will have the following attributes:
- name: The name of the virtual environment.
- installation_folder: The folder where the virtual environment will be installed.
- venv_root_path: The path to the root folder of the virtual environment.
- venv_python_path: The path to the python executable of the virtual environment.

Note: The class will use the run_cmd function that we defined earlier.

In [ ]:
class PyVenv:
    VENV_SCRIPTS_FOLDER_BY_OS = {
        "win32" : r"{}\Scripts",
        "linux" : "{}/bin",
        "darwin": "{}/bin",
    }
    
    def __init__(self, name: str = "venv", installation_folder: str = "."):
        self.name = name
        self.installation_folder = installation_folder

    @property
    def venv_root_path(self):
        return os.path.join(self.installation_folder, self.name)

    @property
    def venv_python_path(self):
        return os.path.join(self.installation_folder, self.VENV_SCRIPTS_FOLDER_BY_OS[sys.platform].format(self.name), "python")

    def create(self):
        run_cmd(f"python -m venv {self.venv_root_path}")

    def run_py_cmd(self, cmd: str):
        return run_cmd(f"{self.venv_python_path} {cmd}")

    def pip_install(self, pkg: str):
        return self.run_py_cmd(f"-m pip install {pkg}")

    def get_pip_list(self):
        return self.run_py_cmd(f"-m pip list")

    def __repr__(self):
        return f"{self.name}@{self.installation_folder}"

We can now create two virtual environments using the class. You can see that the class is more convenient than before.

In [ ]:
venv2 = PyVenv("venv2")
print(f"{venv2 = }")
venv3 = PyVenv("venv3")
print(f"{venv3 = }")

We can now create the virtual environments using the create method.

In [ ]:
venv2.create()

In [ ]:
venv3.create()

We can now install pandas in venv2 and scikit-learn in venv3.

In [ ]:
venv2.get_pip_list()

In [ ]:
venv3.get_pip_list()

In [ ]:
venv2.pip_install("pandas")

In [ ]:
venv3.pip_install("scikit-learn")

In [ ]:
venv2.get_pip_list()

In [ ]:
venv3.get_pip_list()

We can now run the python files using the virtual environments.We will indeed get the same results as before.

In [ ]:
venv2.run_py_cmd("main_using_pandas.py")
venv2.run_py_cmd("main_using_sklearn.py")

In [ ]:
venv3.run_py_cmd("main_using_pandas.py")
venv3.run_py_cmd("main_using_sklearn.py")

## Conclusion

In this tutorial, we learned how to create and manage virtual environments for Python projects. We also learned how to create a class that will help us manage virtual environments. We hope that you learned something new and that you will use virtual environments in your future projects.